<a href="https://colab.research.google.com/github/MinakoNG63/DSFB/blob/main/10_Data_Collection%2C_JSON_and_XML_APIs%2C_BeautifulSoup%2C_and_SQLAlchemy_63070240.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Collection & Data Formats
## JSON, XML APIs, BeautifulSoup and SQLAlchemy (Updated 2022)

Term 1 2022 - Instructor: Teerapong Leelanupab

Teaching Assistant:
1. Piyawat Chuangkrud (Sam)
2. Suvapat Manu (Mint)

***

## Downloading Data
There are several built-in Python modules for performing URL requests.

### Python 2
**urllib**, **urllib2**

1. *urllib* and *urllib2* are both Python modules that do URL request related stuff but offer different functionalities.

    - *urllib2* can accept a Request object to set the headers for a URL request, *urllib* accepts **only** a URL.

    - *urllib* provides the **urlencode** method which is used for the generation of GET query strings, *urllib2* doesn't have such a function. This is one of the reasons why *urllib* is often used along with *urllib2*.
    
### Python 3
**urllib**, **urllib3** and **requests**

1. The built-in Python *urllib.request* module has functions which help in downloading content from HTTP URLs using minimal code. There is no more *urlib2* because the *urllib2* module has been split across several modules in Python 3 named *urllib.request* and *urllib.error*. The Python 2 to Python 3 tool will automatically adapt imports when converting your sources to Python 3.

2. *urllib3* is a third-party package. Despite the name, it is unrelated to the standard library packages, and there is no intention to include it in the standard library in the future.

3. *requests* is built on top of *urllib3*. *requests* internally uses urllib3 with an aim for an easier-to-use API.

    - Python Requests encodes the parameters automatically so you just pass them as simple arguments, unlike in the case of *urllib*, where you need to use the method *urllib.encode()* to encode the parameters before passing them.

    - It automatically decoded the response into Unicode.

    - Requests also has far more convenient error handling. If your authentication failed, urllib would raise a urllib.URLError, while Requests would return a normal response object, as expected. All you have to see if the request was successful by boolean response.ok

**However, directly using *urllib3* can be more efficient, because it lets you reuse the session, whereas requests creates one for every request**

In [ ]:
!pip install urllib3 requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [ ]:
#https://gist.github.com/andrewwatts/2012630
#https://stackoverflow.com/questions/18061640/ignore-certificate-validation-with-urllib3?fbclid=IwAR3S0lDEVJKgO44qKaQ_MlGHk7kPiOP0fwTLLFh7DTE4wyT6YVvwZd30xjg

import time

_URL = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/kmitl.txt"
_NUMBER = 10 #modify this from 1 to 1000 times of attempts

def test_urllib():
  from urllib import request, error
  try:
      response = request.urlopen(_URL)
  except error.HTTPError as e:
      response = e
  response.code
  return response.read()

def test_urllib3():
  import urllib3
  import ssl

  ssl_verify = False
  if (ssl_verify):
      cert_reqs = ssl.CERT_REQUIRED
  else:
      cert_reqs = ssl.CERT_NONE
      urllib3.disable_warnings()

  http = urllib3.PoolManager(cert_reqs = cert_reqs)
  response = http.request('GET', _URL)
  response.status
  return response.data

def test_requests_text():
  import requests
  response = requests.get(_URL)
  response.status_code
  return response.text

def test_requests_content():
  import requests
  response = requests.get(_URL)
  response.status_code
  return response.content

In [ ]:
def main():
  from timeit import Timer
  t_urllib = Timer("test_urllib()", "from __main__ import test_urllib")
  print('{0} urllib: {1}'.format(_NUMBER,  t_urllib.timeit(number=_NUMBER) ) )
  t_urllib3 = Timer("test_urllib3()", "from __main__ import test_urllib3")
  print('{0} urllib3: {1}'.format(_NUMBER,  t_urllib3.timeit(number=_NUMBER)))
  t_requests_text = Timer("test_requests_text()", "from __main__ import test_requests_text")
  print('{0} requests_text: {1}'.format(_NUMBER,  t_requests_text.timeit(number=_NUMBER)))
  t_requests_content = Timer("test_requests_content()", "from __main__ import test_requests_content")
  print('{0} requests_content: {1}'.format(_NUMBER,  t_requests_content.timeit(number=_NUMBER)))

if __name__ == '__main__':
  main()

10 urllib: 10.934259912000016
10 urllib3: 12.241627262000009
10 requests_text: 11.546980679
10 requests_content: 12.637256039999954


## Choice to select which package
1. So, if you just want a simplicity of usage -> go for *requests* #เน้นง่ายในการใช้งาน
2. If you have restriction about installation of other packages -> use standard package, *urllib*
3. If your speed is your concern -> go for *urllib3* #เน้น speed

## 1. Work with *urllib*

In [ ]:
import urllib
import urllib.request
url = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/kmitl.txt"
response = urllib.request.urlopen(url)
text = response.read().decode()
print(text)

	The establishment of the Nondhaburi Telecommunication Training Center on August 24, 1960 with academic cooperation from the Government of Japan marked the origination of KMITL. The training center became the Nodhaburi Institute of Telecommunications under the Columbo Plan, later in 1964.
	As specified by the 1971 King Mongkut's Institute of Technology Act, KMITL was originated by an amalgamation of three technical colleges: Nondhaburi Institute of Telecommunications, North Bangkok Technical College and Thonburi Technical College. In the same year, the Nondhaburi Institute of Telecommunications, or known as King Mongkut's Institute of Technology at Nondhaburi Campus, was relocated to the district of Ladkrabang in Bangkok. The new campus was called ''Chao Khun Taharn Ladkrabang Campus''. The Nondhaburi Institute of Telecommunications became the Faculty of Engineering in 1972. In the same year, the College of Design and Construction located at the Bangplad district was transformed into t

### Wrap your code in Try Except
In practice, we may often want to wrap code to fetch URLs in a try block, to handle the case where we cannot access the URL.

In [ ]:
url = "https://somemissinglink.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/kmitl.txt"
try:
  response = urllib.request.urlopen(url)
  text = response.read().decode()
except:
  print("Failed to retrieve %s" % url)

Failed to retrieve https://somemissinglink.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/kmitl.txt


## 2. Work with *requests*

In [ ]:
import requests
url = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/kmitl.txt"
response = requests.get(url)
# print(response.status_code)
print(response.text)

	The establishment of the Nondhaburi Telecommunication Training Center on August 24, 1960 with academic cooperation from the Government of Japan marked the origination of KMITL. The training center became the Nodhaburi Institute of Telecommunications under the Columbo Plan, later in 1964.
	As specified by the 1971 King Mongkut's Institute of Technology Act, KMITL was originated by an amalgamation of three technical colleges: Nondhaburi Institute of Telecommunications, North Bangkok Technical College and Thonburi Technical College. In the same year, the Nondhaburi Institute of Telecommunications, or known as King Mongkut's Institute of Technology at Nondhaburi Campus, was relocated to the district of Ladkrabang in Bangkok. The new campus was called ''Chao Khun Taharn Ladkrabang Campus''. The Nondhaburi Institute of Telecommunications became the Faculty of Engineering in 1972. In the same year, the College of Design and Construction located at the Bangplad district was transformed into t

In [ ]:
url = "https://somemissinglink.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/kmitl.txt"
try:
  response = requests.get(url)
  response.status_code
  response.text
except:
  print("Failed to retrieve %s" % url)

Failed to retrieve https://somemissinglink.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/kmitl.txt


## 3. Working with CSV

The CSV ("Comma Separated Values") file format is often used to exchange tabular data between different applications, like Excel. Essentially a CSV file is a plain text file where values are split by a comma separator. Alternatively can be tab or space separated.

We could download a CSV file using *urllib.request* and manually parse it...

In [ ]:
# Download the CSV and store as a string
url = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/goal_scorers.csv"
response = urllib.request.urlopen(url)
raw_csv = response.read().decode()
# Parse each line
lines = raw_csv.split("\n")
for l in lines:
  l = l.strip()
  if len(l) > 0:
    # split based on a comma separator
    parts = l.split(",")
    print(parts)

['Player', 'Team', 'Total Goals', 'Penalties', 'Home Goals', 'Away Goals']
['J Vardy', 'Leicester City', '19', '4', '11', '8']
['H Kane', 'Tottenham', '16', '4', '7', '9']
['R Lukaku', 'Everton', '16', '1', '8', '8']
['O Ighalo', 'Watford', '15', '0', '8', '7']
['S Aguero', 'Manchester City', '14', '1', '10', '4']
['R Mahrez', 'Leicester City', '14', '4', '4', '10']
['O Giroud', 'Arsenal', '12', '0', '4', '8']
['D Costa', 'Chelsea', '10', '0', '7', '3']
['J Defoe', 'Sunderland', '10', '0', '3', '7']
['G Wijnaldum', 'Newcastle Utd', '9', '0', '9', '0']
['T Deeney', 'Watford', '8', '5', '2', '6']
['R Barkley', 'Everton', '8', '2', '5', '3']
['A Ayew', 'Swansea City', '8', '0', '5', '3']
['G Sigurdsson', 'Swansea City', '7', '3', '2', '5']
['W Rooney', 'Manchester Utd', '7', '1', '3', '4']
['A Martial', 'Manchester Utd', '7', '0', '4', '3']
['D Alli', 'Tottenham', '7', '0', '1', '6']
['D Payet', 'West Ham Utd', '7', '0', '3', '4']
['M Arnautovic', 'Stoke City', '7', '2', '4', '3']
['Y Tou

### This is what we commonly use to download csv using *pandas*
But we can also use Pandas to directly download and parse CSV data for us, to create a Data Frame which is ready to analyse.

In [ ]:
import pandas as pd
df = pd.read_csv("https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/goal_scorers.csv")
df

,Player,Team,Total Goals,Penalties,Home Goals,Away Goals
0,J Vardy,Leicester City,19,4,11,8
1,H Kane,Tottenham,16,4,7,9
2,R Lukaku,Everton,16,1,8,8
3,O Ighalo,Watford,15,0,8,7
4,S Aguero,Manchester City,14,1,10,4
5,R Mahrez,Leicester City,14,4,4,10
6,O Giroud,Arsenal,12,0,4,8
7,D Costa,Chelsea,10,0,7,3
8,J Defoe,Sunderland,10,0,3,7
9,G Wijnaldum,Newcastle Utd,9,0,9,0


## Working with JSON

[JSON](http://json.org/) is a lightweight format which is becoming increasingly popular for online data exchanged. Based originally on the JavaScript language and (relatively) easy for humans to read and write

The built-in module *json* provides an easy way to encode and decode data in JSON in Python.

In [ ]:
import json

Let's try downloading and parsing a simple JSON file which contains information about a number of books, originally from librarything.com:

In [ ]:
url = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/books.json"
response = urllib.request.urlopen(url)
raw_json = response.read().decode("utf-8")

In [ ]:
print(raw_json)

[{
	"book_id": "13585350",
	"title": "The World Treasury of Science Fiction",
	"ISBN": "",
	"year": 1989,
	"rating": 3,
	"language": "eng"
}, {
	"book_id": "124205572",
	"title": "The War of the Worlds",
	"ISBN": "1936594056",
	"year": 2013,
	"rating": 4,
	"language": "eng"
}, {
	"book_id": "127360065",
	"title": "Under the Dome: A Novel",
	"ISBN": "1439149038",
	"year": 2013,
	"rating": 2,
	"language": "eng"
}, {
	"book_id": "13908800",
	"title": "The Ultimate Hitchhiker's Guide to the Galaxy",
	"ISBN": "0345453743",
	"year": 2002,
	"rating": 5,
	"language": "eng"
}, {
	"book_id": "123734934",
	"title": "The Time Traveler's Wife",
	"ISBN": "1476764832",
	"year": 2014,
	"rating": 5,
	"language": "eng"
}, {
	"book_id": "13603020",
	"title": "Salem's Lot",
	"ISBN": "0451098277",
	"year": 1976,
	"rating": 3,
	"language": "eng"
}, {
	"book_id": "124173974",
	"title": "Republic",
	"ISBN": "039395501X",
	"year": 1985,
	"rating": 3,
	"language": "eng"
}, {
	"book_id": "123102859",
	"title": "

We can now parse the JSON, converting it from a string into a useful Python data structure:

In [ ]:
data = json.loads(raw_json)
print(data)

[{'book_id': '13585350', 'title': 'The World Treasury of Science Fiction', 'ISBN': '', 'year': 1989, 'rating': 3, 'language': 'eng'}, {'book_id': '124205572', 'title': 'The War of the Worlds', 'ISBN': '1936594056', 'year': 2013, 'rating': 4, 'language': 'eng'}, {'book_id': '127360065', 'title': 'Under the Dome: A Novel', 'ISBN': '1439149038', 'year': 2013, 'rating': 2, 'language': 'eng'}, {'book_id': '13908800', 'title': "The Ultimate Hitchhiker's Guide to the Galaxy", 'ISBN': '0345453743', 'year': 2002, 'rating': 5, 'language': 'eng'}, {'book_id': '123734934', 'title': "The Time Traveler's Wife", 'ISBN': '1476764832', 'year': 2014, 'rating': 5, 'language': 'eng'}, {'book_id': '13603020', 'title': "Salem's Lot", 'ISBN': '0451098277', 'year': 1976, 'rating': 3, 'language': 'eng'}, {'book_id': '124173974', 'title': 'Republic', 'ISBN': '039395501X', 'year': 1985, 'rating': 3, 'language': 'eng'}, {'book_id': '123102859', 'title': 'The Road', 'ISBN': '0307387895', 'year': 2006, 'rating': 5,

We can now iterate through the books in the list and extract the relevant information that we require.

In [ ]:
for book in data:
  print( "%s = %d" % ( book["title"], book["year"] ) )

The World Treasury of Science Fiction = 1989
The War of the Worlds = 2013
Under the Dome: A Novel = 2013
The Ultimate Hitchhiker's Guide to the Galaxy = 2002
The Time Traveler's Wife = 2014
Salem's Lot = 1976
Republic = 1985
The Road = 2006


We then use json_normalize in Pandas to create a Data Frame of semi-structured JSON data to make it ready to analyse.

In [ ]:
from pandas import json_normalize

df = json_normalize(data)
df.head(5)

,book_id,title,ISBN,year,rating,language
0,13585350,The World Treasury of Science Fiction,,1989,3,eng
1,124205572,The War of the Worlds,1936594056,2013,4,eng
2,127360065,Under the Dome: A Novel,1439149038,2013,2,eng
3,13908800,The Ultimate Hitchhiker's Guide to the Galaxy,0345453743,2002,5,eng
4,123734934,The Time Traveler's Wife,1476764832,2014,5,eng


### OR
Alternatively, we can also use Pandas to directly download and parse JSON data for us, to create a Data Frame which is ready to analyse.

In [ ]:
import pandas as pd
link = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/books.json"
df = pd.read_json( link, orient="records")
df.head(5)

,book_id,title,ISBN,year,rating,language
0,13585350,The World Treasury of Science Fiction,,1989,3,eng
1,124205572,The War of the Worlds,1936594056,2013,4,eng
2,127360065,Under the Dome: A Novel,1439149038,2013,2,eng
3,13908800,The Ultimate Hitchhiker's Guide to the Galaxy,0345453743,2002,5,eng
4,123734934,The Time Traveler's Wife,1476764832,2014,5,eng


## Working with XML

Extensible Markup Language (XML) is a markup language that defines a set of rules for encoding documents in a format which is both human-readable and machine-readable. XML is a widely-adopted format. Python includes several built-in modules for parsing XML data.

The *xml.etree.ElementTree* module can be used to extract data from a simple XML file based on its tree structure.

In [ ]:
# download the content
url = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/books.xml"
response = urllib.request.urlopen(url)
raw_xml = response.read().decode()
print(raw_xml)

<?xml version="1.0" encoding="UTF-8"?>
<booklist>
   <book id="13585350">
      <title>The World Treasury of Science Fiction</title>
      <ISBN />
      <year>1989</year>
      <rating>3</rating>
      <language>eng</language>
   </book>
   <book id="124205572">
      <title>The War of the Worlds</title>
      <ISBN>1936594056</ISBN>
      <year>2013</year>
      <rating>4</rating>
      <language>eng</language>
   </book>
   <book id="127360065">
      <title>Under the Dome: A Novel</title>
      <ISBN>1439149038</ISBN>
      <year>2013</year>
      <rating>2</rating>
      <language>eng</language>
   </book>
   <book id="13908800">
      <title>The Ultimate Hitchhiker's Guide to the Galaxy</title>
      <ISBN>0345453743</ISBN>
      <year>2002</year>
      <rating>5</rating>
      <language>eng</language>
   </book>
   <book id="123734934">
      <title>The Time Traveler's Wife</title>
      <ISBN>1476764832</ISBN>
      <year>2014</year>
      <rating>5</rating>
      <language>eng

We can use the *xml.etree.ElementTree.fromstring()* function to parse content from a string containing XML data.

In [ ]:
import xml.etree.ElementTree as et
xroot = et.fromstring(raw_xml)

NameError: ignored

An XML tree has a root node (i.e. the top level of the document), with child nodes at lower levels. We can iterate over these:

In [ ]:
for child in xroot:
  # get the name of the tag, along with any XML attributes which the tag has
  print( child.tag, child.attrib )

book {'id': '13585350'}
book {'id': '124205572'}
book {'id': '127360065'}
book {'id': '13908800'}
book {'id': '123734934'}
book {'id': '13603020'}
book {'id': '124173974'}
book {'id': '123102859'}


We can also query to find tags with specific names, such as '<book>' and then in turn find child nodes of that tag with a specific name.

In [ ]:
for book in xroot.findall("book"):
  # get the text inside a <title> tag, contained within a <book> tag
  title = book.find("title").text
  print(title)

The World Treasury of Science Fiction
The War of the Worlds
Under the Dome: A Novel
The Ultimate Hitchhiker's Guide to the Galaxy
The Time Traveler's Wife
Salem's Lot
Republic
The Road


We can parse xml to Pandas dataframes, which is ready to analyse.

In [ ]:
df_cols = ["id", "title", "ISBN", "year", "rating", "language"]
df = pd.DataFrame(columns = df_cols)

for node in xroot:
  s_id = node.attrib.get("id")
  s_title = node.find("title").text
  s_isbn = node.find("ISBN").text
  s_year = node.find("year").text
  s_rating = node.find("rating").text
  s_language = node.find("language").text

  #print("%s\t%s\t%s\t%s\t%s\t%s " % (s_id, s_title, s_isbn, s_year, s_rating, s_language))
  df = df.append(pd.Series([s_id, s_title, s_isbn, s_year, s_rating, s_language],
                              index = df_cols),
                              ignore_index=True)

df

,id,title,ISBN,year,rating,language
0,13585350,The World Treasury of Science Fiction,None,1989,3,eng
1,124205572,The War of the Worlds,1936594056,2013,4,eng
2,127360065,Under the Dome: A Novel,1439149038,2013,2,eng
3,13908800,The Ultimate Hitchhiker's Guide to the Galaxy,0345453743,2002,5,eng
4,123734934,The Time Traveler's Wife,1476764832,2014,5,eng
5,13603020,Salem's Lot,0451098277,1976,3,eng
6,124173974,Republic,039395501X,1985,3,eng
7,123102859,The Road,0307387895,2006,5,eng


## Working with HTML

[HyperText Markup Language (HTML)](https://en.wikipedia.org/wiki/HTML) is a language that web pages are created in. HTML isn’t a programming language, like Python — instead, it’s a markup language that tells a browser how to layout content. HTML allows you to do similar things to what you do in a word processor like Microsoft Word — make text bold, create paragraphs, and so on. Because HTML isn’t a programming language, it isn’t nearly as complex as Python.

The built-in Python urllib.request module has functions which help in downloading content from HTTP URLs using minimal code:

In [ ]:
import urllib.request
link = "https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/sample_web/sample.html"
response = urllib.request.urlopen(link)
html = response.read().decode()

We can simple use the for-loop to read the html file line by line to see its structure.

In [ ]:
lines = html.strip().split("\n")
for l in lines:
  print(l)

<html>
  <head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  <title> KMITL Faculty/College</title>
  <link href="./css/sample.css" rel="stylesheet" type="text/css">
</head>
<body>
  <div class="topbar">
    <div id="top-logo2f">
      <img src="http://kmitl.ac.th/frontend/images/logo.png" alt="">
    </div>
    <div id="top-text2f">
      <h1>King Mongkut's Institute of Technology Ladkrabang</h1>
    </div>
  </div>

  <div id="main" style="margin: 20px;">
    <h1>Faculty/College in KMITL</h1>
    <div id="content">
        <h3><a href="http://www.ceir.kmitl.ac.th">College of Educational Innovation Research</a></h3>
        <h3><a href="http://www.music-engineering.kmitl.ac.th/">Institute of Music Science and Engineering</a></h3>
        <h3><a href="http://www.kosen.kmitl.ac.th/">KOSEN-KMITL</a></h3>
        <h3><a href="http://www.chumphon.kmitl.ac.th">KMITL Prince of Chumphon Campus </a></h3>
        <h3><a href="http://engineer.kmitl.ac.th/">Faculty of Engin

### The requests library

The first thing we’ll need to do to scrape a web page is to download the page. We can download pages using the Python [requests](https://2.python-requests.org//en/master/) library. The requests library will make a `GET` request to a web server, which will download the HTML contents of a given web page for us. There are several different types of `requests` we can make using requests, of which `GET` is just one. If you want to learn more, check out this tutorial for using [API](https://www.dataquest.io/blog/python-api-tutorial/) requests in Python.

Let’s try downloading a simple sample website, [https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week4/sample_web/sample.html](https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week4/sample_web/sample.html). We’ll need to first download it using the requests.get method.

In [ ]:
import requests
page = requests.get("https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/sample_web/sample.html")
page

<Response [200]>

After running our request, we get a [Response](https://2.python-requests.org//en/master/user/quickstart/#response-content) object. This object has a `status_code` property, which indicates if the page was downloaded successfully:

In [ ]:
page.status_code

200

A `status_code` of `200` means that the page downloaded successfully. We won’t fully dive into status codes here, but a status code starting with a `2` generally indicates success, and a code starting with a `4` or a `5` indicates an error.

We can print out the HTML content of the page using the content property:

In [ ]:
page.content

b'<html>\n  <head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n  <title> KMITL Faculty/College</title>\n  <link href="./css/sample.css" rel="stylesheet" type="text/css">\n</head>\n<body>\n  <div class="topbar">\n    <div id="top-logo2f">\n      <img src="http://kmitl.ac.th/frontend/images/logo.png" alt="">\n    </div>\n    <div id="top-text2f">\n      <h1>King Mongkut\'s Institute of Technology Ladkrabang</h1>\n    </div>\n  </div>\n\n  <div id="main" style="margin: 20px;">\n    <h1>Faculty/College in KMITL</h1>\n    <div id="content">\n        <h3><a href="http://www.ceir.kmitl.ac.th">College of Educational Innovation Research</a></h3>\n        <h3><a href="http://www.music-engineering.kmitl.ac.th/">Institute of Music Science and Engineering</a></h3>\n        <h3><a href="http://www.kosen.kmitl.ac.th/">KOSEN-KMITL</a></h3>\n        <h3><a href="http://www.chumphon.kmitl.ac.th">KMITL Prince of Chumphon Campus </a></h3>\n        <h3><a href="http://engineer.kmitl

## Parsing a page with BeautifulSoup

As you can see above, we now have downloaded an HTML document.

We can use the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) library to parse this document, and extract the text from the `h3` tag. We first have to import the library, and create an instance of the `BeautifulSoup` class to parse our document:

### Parser in Beautiful Soup
Beautiful Soup supports many parsers, e.g.

**html.parser** - `BeautifulSoup(markup, "html.parser")`

- Advantages: Batteries included, Decent speed, Lenient (as of Python 2.7.3 and 3.2.)

- Disadvantages: Not very lenient (before Python 2.7.3 or 3.2.2)

**lxml** - `BeautifulSoup(markup, "lxml")`

- Advantages: Very fast, Lenient

- Disadvantages: External C dependency

**html5lib** - `BeautifulSoup(markup, "html5lib")`

- Advantages: Extremely lenient, Parses pages the same way a web browser does, Creates valid HTML5

- Disadvantages: Very slow, External Python dependency

### Example 1 - parsing html from an simple page from my website
Extract all contents within tag `<h3>`

In [ ]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import bs4
soup = bs4.BeautifulSoup(page.content, 'html.parser')

for match in soup.find_all("h3"):
  text = match
  print(text)

<h3><a href="http://www.ceir.kmitl.ac.th">College of Educational Innovation Research</a></h3>
<h3><a href="http://www.music-engineering.kmitl.ac.th/">Institute of Music Science and Engineering</a></h3>
<h3><a href="http://www.kosen.kmitl.ac.th/">KOSEN-KMITL</a></h3>
<h3><a href="http://www.chumphon.kmitl.ac.th">KMITL Prince of Chumphon Campus </a></h3>
<h3><a href="http://engineer.kmitl.ac.th/">Faculty of Engineering</a></h3>
<h3><a href="http://www.arch.kmitl.ac.th/">Faculty of Architecture</a></h3>
<h3><a href="http://www.science.kmitl.ac.th/main.php">Faculty of Science</a></h3>
<h3><a href="http://www.ietech.kmitl.ac.th">Faculty of Industrial Education and Technology</a></h3>
<h3><a href="http://www.agri.kmitl.ac.th/AgriTH/">Faculty of Agricultural Technology</a></h3>
<h3><a href="http://www.it.kmitl.ac.th/">Faculty of Information Technology</a></h3>
<h3><a href="http://www.agroind.kmitl.ac.th/th/home-agro">Faculty of Agro-Industry</a></h3>
<h3><a href="http://www.fam.kmitl.ac.th/">

### Example 2 - extract content from all divs in a kmitl history page
Extract all contents within tag `<div>`

In [ ]:
url = 'http://www.kmitl.ac.th/en/detail/2016-12-21-22-32-22'
page = requests.get(url)
page

# Consise our code by import BeautifulSoup without necessarily calling bs4 again
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, "html5lib")
soup.title

<title>Page not found | สถาบันเทคโนโลยีพระจอมเกล้าเจ้าคุณทหารลาดกระบัง</title>

In [ ]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-41909381-25">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];function gtag(){dataLayer.push(arguments)};gtag("js", new Date());gtag("config", "UA-41909381-25", {"groups":"default","anonymize_ip":true,"page_path":"/404.html?page=" + document.location.pathname + document.location.search + "&from=" + document.referrer});
  </script>
  <link href="https://www.kmitl.ac.th/" rel="canonical"/>
  <link href="https://www.kmitl.ac.th/" rel="shortlink"/>
  <meta co

In [ ]:
contentDivs = soup.findAll("div", {"class": "pbox"})
contentDivs

[]

In [ ]:
i = 1
texts = []
for contentDiv in contentDivs:
  text = contentDiv.text.strip()
  if len(text) > 0:
    print("---  %d ------" % i)
    texts.append(text)
    print(text)
    i += 1

## Working with APIs

### Example 1 - Wikipedia

As a simple example of using an Online API, we will retrieve JSON data from the Wikipedia web API. The Wikipedia page for 'KMITL' is [here](https://en.wikipedia.org/wiki/King_Mongkut%27s_Institute_of_Technology_Ladkrabang). We can retrieve this data in a cleaner JSON format from the Wikipedia API endpoint (https://en.wikipedia.org/w/api.php).

In [ ]:
title = "King_Mongkut%27s_Institute_of_Technology_Ladkrabang"
url = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=true&titles=" + title
print(url)

https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=true&titles=King_Mongkut%27s_Institute_of_Technology_Ladkrabang


In [ ]:
response = urllib.request.urlopen(url)
raw_json = response.read().decode("utf-8")

Once we have downloaded the JSON data into a string, we parse it using the *loads()* function, which will convert it into an actual Python dictionary.

In [ ]:
data = json.loads(raw_json)
data

{'batchcomplete': '',
 'warnings': {'extracts': {'*': 'HTML may be malformed and/or unbalanced and may omit inline images. Use at your own risk. Known problems are listed at https://www.mediawiki.org/wiki/Special:MyLanguage/Extension:TextExtracts#Caveats.'}},
 'query': {'normalized': [{'from': "King_Mongkut's_Institute_of_Technology_Ladkrabang",
    'to': "King Mongkut's Institute of Technology Ladkrabang"}],
  'pages': {'1232312': {'pageid': 1232312,
    'ns': 0,
    'title': "King Mongkut's Institute of Technology Ladkrabang",
    'extract': "<p><b>King Mongkut's Institute of Technology Ladkrabang</b> (<b>KMITL</b> or <b>KMIT Ladkrabang</b> for short) is a research and educational institution in Thailand. It is situated in Lat Krabang District, Bangkok approximately 30\xa0km east of the city center.  The university consists of nine faculties: engineering, architecture, science, industrial education and technology, agricultural technology, information technology, food industry, libera

The response still needs to be inspected. Note that the results we want are are in *data["query"]["pages"]*:

In [ ]:
print(data["query"]["pages"])

{'1232312': {'pageid': 1232312, 'ns': 0, 'title': "King Mongkut's Institute of Technology Ladkrabang", 'extract': "<p><b>King Mongkut's Institute of Technology Ladkrabang</b> (<b>KMITL</b> or <b>KMIT Ladkrabang</b> for short) is a research and educational institution in Thailand. It is situated in Lat Krabang District, Bangkok approximately 30\xa0km east of the city center.  The university consists of nine faculties: engineering, architecture, science, industrial education and technology, agricultural technology, information technology, food industry, liberal arts, and medicine.\n</p>"}}


In [ ]:
result = data["query"]["pages"]["1232312"]
print(result["title"])
print(result["extract"])

King Mongkut's Institute of Technology Ladkrabang
<p><b>King Mongkut's Institute of Technology Ladkrabang</b> (<b>KMITL</b> or <b>KMIT Ladkrabang</b> for short) is a research and educational institution in Thailand. It is situated in Lat Krabang District, Bangkok approximately 30 km east of the city center.  The university consists of nine faculties: engineering, architecture, science, industrial education and technology, agricultural technology, information technology, food industry, liberal arts, and medicine.
</p>


### Example 2 - Currency Exchange Rates

In the next example, we will use the *Fixer.io* API to get currency exchange rate information: http://fixer.io

For API documentation: https://fixer.io/documentation

To retrieve all rates in EUROs, we retrieve the following:

In [ ]:
ACCESS_KEY = "0c9904dea3d2c46b78686bc16bbba722"

In [ ]:
url = "http://data.fixer.io/api/latest?access_key=" + ACCESS_KEY
response = urllib.request.urlopen(url)
raw_json = response.read().decode("utf-8")
print(raw_json)

{"success":true,"timestamp":1661484846,"base":"EUR","date":"2022-08-26","rates":{"AED":3.662765,"AFN":88.283541,"ALL":116.825255,"AMD":403.280144,"ANG":1.799996,"AOA":428.019767,"ARS":137.007037,"AUD":1.431896,"AWG":1.799987,"AZN":1.697724,"BAM":1.957239,"BBD":2.016572,"BDT":94.906107,"BGN":1.954996,"BHD":0.376022,"BIF":2059.742466,"BMD":0.997223,"BND":1.388299,"BOB":6.901194,"BRL":5.096106,"BSD":0.998729,"BTC":4.623666e-5,"BTN":79.73649,"BWP":12.739369,"BYN":2.520902,"BYR":19545.5656,"BZD":2.013139,"CAD":1.291249,"CDF":2014.389342,"CHF":0.961263,"CLF":0.032472,"CLP":896.007449,"CNY":6.835364,"COP":4381.726891,"CRC":633.976294,"CUC":0.997223,"CUP":26.426402,"CVE":110.34165,"CZK":24.658926,"DJF":177.799751,"DKK":7.437666,"DOP":53.088209,"DZD":140.69244,"EGP":19.140861,"ERN":14.958341,"ETB":52.684719,"EUR":1,"FJD":2.196084,"FKP":0.821318,"GBP":0.843132,"GEL":2.83709,"GGP":0.821318,"GHS":9.912386,"GIP":0.821318,"GMD":54.147238,"GNF":8617.20209,"GTQ":7.727668,"GYD":208.945147,"HKD":7.82398

Parse the JSON data

In [ ]:
data = json.loads(raw_json)
# List all the rates
data

{'success': True,
 'timestamp': 1661484846,
 'base': 'EUR',
 'date': '2022-08-26',
 'rates': {'AED': 3.662765,
  'AFN': 88.283541,
  'ALL': 116.825255,
  'AMD': 403.280144,
  'ANG': 1.799996,
  'AOA': 428.019767,
  'ARS': 137.007037,
  'AUD': 1.431896,
  'AWG': 1.799987,
  'AZN': 1.697724,
  'BAM': 1.957239,
  'BBD': 2.016572,
  'BDT': 94.906107,
  'BGN': 1.954996,
  'BHD': 0.376022,
  'BIF': 2059.742466,
  'BMD': 0.997223,
  'BND': 1.388299,
  'BOB': 6.901194,
  'BRL': 5.096106,
  'BSD': 0.998729,
  'BTC': 4.623666e-05,
  'BTN': 79.73649,
  'BWP': 12.739369,
  'BYN': 2.520902,
  'BYR': 19545.5656,
  'BZD': 2.013139,
  'CAD': 1.291249,
  'CDF': 2014.389342,
  'CHF': 0.961263,
  'CLF': 0.032472,
  'CLP': 896.007449,
  'CNY': 6.835364,
  'COP': 4381.726891,
  'CRC': 633.976294,
  'CUC': 0.997223,
  'CUP': 26.426402,
  'CVE': 110.34165,
  'CZK': 24.658926,
  'DJF': 177.799751,
  'DKK': 7.437666,
  'DOP': 53.088209,
  'DZD': 140.69244,
  'EGP': 19.140861,
  'ERN': 14.958341,
  'ETB': 52.68

In [ ]:
# Get a specific rate
data["rates"]["CHF"]

0.961263

We can change the URL to get rates for a different currency, such as US Dollars (USD):

In [ ]:
url = "http://data.fixer.io/api/latest?access_key=" + ACCESS_KEY + "&symbols=THB"
print(url)
# Retrieve the JSON
response = urllib.request.urlopen(url)
raw_json = response.read().decode("utf-8")
# Parse the JSON
data = json.loads(raw_json)
# Display the rates data for US dollars
data["rates"]

http://data.fixer.io/api/latest?access_key=0c9904dea3d2c46b78686bc16bbba722&symbols=THB


{'THB': 35.762116}

In [ ]:
data

{'success': True,
 'timestamp': 1661488144,
 'base': 'EUR',
 'date': '2022-08-26',
 'rates': {'THB': 35.762116}}

In [ ]:
df = json_normalize(data)
df

,success,timestamp,base,date,rates.THB
0,True,1661488144,EUR,2022-08-26,35.762116


### Example 3 - Thairath with Python requests

Content: [https://www.thairath.co.th](https://www.thairath.co.th)

API: [https://api.thairath.co.th](https://api.thairath.co.th)


Here, we will work with the website Thairath that has been created for recording news in Thairath digital newspaper.




#### **Query example**

https://api.thairath.co.th/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627702200000

1. base_url = ```https://api.thairath.co.th```
2. base_path = ```/v1.1/thairath-online/load-more?path=```
3. category_path = ```business/investment```
4. middle_param = ```&key=latest&ts=```
5. timestamp = ```1627702200000```

#### **what we want to collect here**
* title
* topic
* detail
* date

[Namespace](https://docs.python.org/3/library/argparse.html#argparse.Namespace)

Simple class used by default by parse_args() to create an object holding attributes and return it.

In [ ]:
from argparse import Namespace
args = Namespace()

In [ ]:
args.base_url = 'https://api.thairath.co.th'
args.base_path = '/v1.1/thairath-online/load-more?path='
args.middle_param = '&key=latest&ts='

In [ ]:
args.path = ['business/investment', 'business/economics', 'business/market',
             'business/finance', 'business/feature']

In [ ]:
args.ts = ['1627702200000', '1627694880000', '1627811340000', '1627616820000', '1627613040000']
# args.ts =   ['1661414835000']

In [ ]:
#Test api with only one category and one timestamp
url = f'{args.base_url}{args.base_path}{args.path[0]}{args.middle_param}{args.ts[0]}'
url

'https://api.thairath.co.th/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627702200000'

In [ ]:
import requests
from pandas import json_normalize

In [ ]:
response = requests.get(url)
data = response.json()
data

{'meta': {'title': 'ข่าวการลงทุน ข่าวหุ้นวันนี้ ตลาดหุ้น ข่าวราคาทอง | ไทยรัฐออนไลน์',
  'description': 'ข่าวการลงทุนไทยรัฐออนไลน์ ข่าวหุ้น ข่าวหุ้นวันนี้ ตลาดหุ้นไทย ข่าวราคาทอง 2565 โดยทีมข่าวเศรษฐกิจมืออาชีพไทยรัฐ',
  'canonical': 'https://www.thairath.co.th/sport/footballprogram/laliga',
  'fullPath': 'business/investment',
  'image': 'https://static.thairath.co.th/media/00_A9B32874C1E1280.jpg'},
 'headers': {'Cache-Control': 'public, max-age=300, must-revalidate'},
 'sectionName': 'business',
 'topicName': 'investment',
 'topicNameTh': 'การลงทุน',
 'fullPath': 'business/investment',
 'lastModified': '2022-08-26T04:41:59.056Z',
 'items': {'contents': [{'id': 2153893,
    '_id': '6104b6e2746a1b661ddbaf62',
    'type': 1,
    'title': 'ราคาทองวันนี้ 31/7/64 ล่าสุด เปิดตลาดเช้าวันเสาร์ ปรับลด 100 บาท',
    'credit': 'ไทยรัฐออนไลน์',
    'label': [],
    'section': 'เศรษฐกิจ',
    'sectionEn': 'business',
    'topic': 'ราคาทองคำ',
    'topicEn': 'gold',
    'topicPath': 'business/inves

In [ ]:
topic_list, title_list, abstract_list, url_list, timestamp_list = [], [], [], [], []

for doc in data['items']['contents']:
  topic_list.append('business')
  title_list.append(doc['title'])
  abstract_list.append(doc['abstract'])
  url_list.append(doc['canonical'])
  timestamp_list.append(doc['publishTime'])

In [ ]:
import pandas as pd
df = pd.DataFrame({'topic':topic_list,
                   'title':title_list,
                   'abstract':abstract_list,
                   'url':url_list,
                   'timestamp':timestamp_list
                  })

In [ ]:
df

,topic,title,abstract,url,timestamp
0,business,ราคาทองวันนี้ 31/7/64 ล่าสุด เปิดตลาดเช้าวันเส...,ราคาทองวันนี้ เปิดตลาดเช้าวันเสาร์ ราคาปรับลด ...,https://www.thairath.co.th/business/investment...,2021-07-31T02:44:00.000Z
1,business,ราคาทองคำยังพุ่งต่อ YLG มองเทคนิคระยะสั้นสดใส ...,ราคาทองคำยังพุ่งต่อ รับผลประชุมเฟดที่ยังไม่เร่...,https://www.thairath.co.th/business/investment...,2021-07-31T02:15:00.000Z
2,business,เลือกหุ้นปันผลสูง!!,"ดัชนีหุ้นไทยวันที่ 30 ก.ค.64 ปิดที่ 1,521.92 จ...",https://www.thairath.co.th/business/investment...,2021-07-30T22:01:00.000Z
3,business,"มูลค่าการซื้อขาย 7,946.75 ล้านบาท",ดัชนีเศรษฐกิจ,https://www.thairath.co.th/business/investment...,2021-07-30T22:01:00.000Z
4,business,หุ้นไทยวันนี้ ปิดตลาดหุ้นเช้า ปรับลด 16.45 ดัช...,หุ้นไทยวันนี้ ปิดตลาดหุ้นเช้า ปรับลด 16.45 ดัช...,https://www.thairath.co.th/business/investment...,2021-07-30T05:45:00.000Z
5,business,"ไทยออยล์ ประกาศลงทุนธุรกิจ ""ปิโตรเคมีโอเลฟินส์...",ไทยออยล์ ประกาศลงทุนธุรกิจปิโตรเคมีสายโอเลฟินส...,https://www.thairath.co.th/business/investment...,2021-07-30T04:55:00.000Z
6,business,ราคาทองวันนี้ 30/7/64 ล่าสุด ปรับขึ้น 100 ทองร...,ราคาทองวันนี้ เปิดตลาดเช้าวันศุกร์ ราคาปรับขึ้...,https://www.thairath.co.th/business/investment...,2021-07-30T02:43:00.000Z
7,business,ใช้เงินทำงานผ่านกองทุน Income Fund,Income Fund เป็นกองทุนรวมผสม ลงทุนในตราสารหนี้...,https://www.thairath.co.th/business/investment...,2021-07-29T22:08:00.000Z


In [ ]:
topic_list, title_list, abstract_list, url_list, timestamp_list = [], [], [], [], []

In [ ]:
url = 'https://api.thairath.co.th/v1.1/thairath-online/load-more?path=business/economics&key=latest&ts=1626223320000'
res = requests.get(url)
json_val = res.json()

In [ ]:
# args.path = ['business/feature']
# args.ts = ['1581891780000']

In [ ]:
counter = 0
import time
for path, ts in zip(args.path, args.ts):
  sub_url = args.base_path + path + args.middle_param + ts
  print(sub_url)
  for i in range(10): #only 1 seed ไต่ต่อแค่ 10 หน้า
    print("# Request: {}, {}".format(counter + 1, i + 1))
    print(sub_url)
    url = args.base_url + sub_url
    print(url)
    res = requests.get(url)
    json_val = res.json()

    for doc in json_val['items']['contents']:
      topic_list.append('business')
      title_list.append(doc['title'])
      abstract_list.append(doc['abstract'])
      url_list.append(doc['canonical'])
      timestamp_list.append(doc['publishTime'])

    sub_url = json_val['items']['paging']
    if 'undefined' in sub_url or sub_url == '' or sub_url is None:
      break
    if i%5 == 0:
      time.sleep(1) #คืน 1 วิ

    counter = counter + 1

/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627702200000
# Request: 1, 1
/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627702200000
https://api.thairath.co.th/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627702200000
# Request: 2, 2
/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627596480000
https://api.thairath.co.th/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627596480000
# Request: 3, 3
/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627509660000
https://api.thairath.co.th/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627509660000
# Request: 4, 4
/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627365180000
https://api.thairath.co.th/v1.1/thairath-online/load-more?path=business/investment&key=latest&ts=1627365180000
# Request: 5, 5
/v1.1/thairath-online/load-more?path=business/inves

In [ ]:
import pandas as pd
df = pd.DataFrame({'topic':topic_list,
                   'title':title_list,
                   'abstract':abstract_list,
                   'url':url_list,
                   'timestamp':timestamp_list
                  })

In [ ]:
df.shape
# (111 * 8) + 2 = 890

(800, 5)

In [ ]:
df.head()

,topic,title,abstract,url,timestamp
0,business,ราคาทองวันนี้ 31/7/64 ล่าสุด เปิดตลาดเช้าวันเส...,ราคาทองวันนี้ เปิดตลาดเช้าวันเสาร์ ราคาปรับลด ...,https://www.thairath.co.th/business/investment...,2021-07-31T02:44:00.000Z
1,business,ราคาทองคำยังพุ่งต่อ YLG มองเทคนิคระยะสั้นสดใส ...,ราคาทองคำยังพุ่งต่อ รับผลประชุมเฟดที่ยังไม่เร่...,https://www.thairath.co.th/business/investment...,2021-07-31T02:15:00.000Z
2,business,เลือกหุ้นปันผลสูง!!,"ดัชนีหุ้นไทยวันที่ 30 ก.ค.64 ปิดที่ 1,521.92 จ...",https://www.thairath.co.th/business/investment...,2021-07-30T22:01:00.000Z
3,business,"มูลค่าการซื้อขาย 7,946.75 ล้านบาท",ดัชนีเศรษฐกิจ,https://www.thairath.co.th/business/investment...,2021-07-30T22:01:00.000Z
4,business,หุ้นไทยวันนี้ ปิดตลาดหุ้นเช้า ปรับลด 16.45 ดัช...,หุ้นไทยวันนี้ ปิดตลาดหุ้นเช้า ปรับลด 16.45 ดัช...,https://www.thairath.co.th/business/investment...,2021-07-30T05:45:00.000Z


In [ ]:
df.tail()

,topic,title,abstract,url,timestamp
795,business,"""กิมจิคุณคิม"" กิมจิโอ่งสายพันธุ์ไทย เกิดช่วงโค...","เศรษฐีป้ายแดง คุยกับ ""เยจิน คิม"" เจ้าของร้านกิ...",https://www.thairath.co.th/business/feature/20...,2021-04-30T10:22:00.000Z
796,business,"คุยกับ เจ้าของ ""โตเกียวยักษ์นายเจ"" เริ่มจากทุน...","เศรษฐีป้ายแดง พูดคุยกับ เจ้าของธุรกิจ ""โตเกียว...",https://www.thairath.co.th/business/feature/20...,2021-04-29T10:28:00.000Z
797,business,จับตากระแสร้อนแรงของตลาดคริปโตฯ หลังมูลค่าพุ่ง...,ปรมินทร์ แนะนักลงทุนตลาดคริปโตฯ จับตากระแสหลัง...,https://www.thairath.co.th/business/feature/20...,2021-04-28T07:30:00.000Z
798,business,"เราเป็นมนุษย์ 40 คิดแบบ ""กรมเชษฐ์"" AssetWise ข...",เราเป็นมนุษย์ 40 หรือ ปี 40 ได้เห็นภาพบริษัทห...,https://www.thairath.co.th/business/feature/20...,2021-04-27T01:20:00.000Z
799,business,ก้าวสู่สังคมคาร์บอนต่ำ โอกาสใหม่ของธุรกิจพิชิต...,หลายประเทศทั่วโลกจึงได้มีความพยายามที่จะลดการป...,https://www.thairath.co.th/business/feature/20...,2021-04-26T04:30:00.000Z


### Example 4 - Thaichana with Python requests <font color='red'>(Need to try this in Jupyter Notebook in local machine)</font>

https://www.thaichana.com/

Here, we will work with the website Thaichana that has been created for recording citizen traffics in going in and out shops in Thailand.

https://merchant.thaichana.com/

<figure>
<center>
<img src="https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/images/search_page.jpg" width="400"/>
<figcaption><em>Search Page</em>
</figcaption></center>
</figure>

https://merchant.thaichana.com/search?size=10&page=0&category=%E0%B8%97%E0%B8%B8%E0%B8%81%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B9%80%E0%B8%A0%E0%B8%97&province=%E0%B8%81%E0%B8%A3%E0%B8%B8%E0%B8%87%E0%B9%80%E0%B8%97%E0%B8%9E%E0%B8%A1%E0%B8%AB%E0%B8%B2%E0%B8%99%E0%B8%84%E0%B8%A3&rating=0&passQuestionnaire=false&lockLatLong=false

See example of information from Thaichana.com
The image below show *types* of location.

<figure>
<center>
<img src="https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/images/shop_type.jpg" width="400"/>
<figcaption><em>Shop type</em>
</figcaption></center>
</figure>


The image below show location information, such as The Mall Bangkae.


<figure>
<center>
<img src="https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/images/themall_bangkae.jpg" width="400"/>
<figcaption><em>The Mall Bangkae</em>
</figcaption></center>
</figure>


In [ ]:
import requests
from pandas import json_normalize

url = "https://api-customer.thaichana.com/shop/0001/S0000011223"
response = requests.get(url)
print(type(response))
print(response.text)

<class 'requests.models.Response'>
<!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>403</title>403 Forbidden


In [ ]:
!pip install cfscrape

import cfscrape
session = requests.Session()
session.headers = ...
scraper = cfscrape.create_scraper(sess=session)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
url = "https://api-customer.thaichana.com/shop/0001/S0000011223"
response = scraper.get(url)
print(type(response))
print(response.text)

<class 'requests.models.Response'>
<!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>403</title>403 Forbidden


In [ ]:
url = "https://api-customer.thaichana.com/shop/0001/S0000010001"
response = requests.get(url)
data = response.json()
df = json_normalize(data)
df

JSONDecodeError: ignored

### Define the above code as a fuction for reusable and convenient calls

In [ ]:
def getData(url):
  response = requests.get(url)
  return response.json()

### Try to use the just defined function

In [ ]:
url = "https://api-customer.thaichana.com/shop/0001/S0000010006"
getData(url)

NameError: ignored

### Try to retrieve more data from the same url but multiple times
You can notice that the item is defined by the last route of url.

In [ ]:
shops = []
for i in range(10):
  data = getData(url)
  shops.append(data)
  print(data)

NameError: ignored

In [ ]:
### Let's print details of all shops that we have just gather
df = json_normalize(shops)
df

""


### we need to define another fucntion to customize our url to item, from a given base url.

1. pad zeros to a string of number
> n = '4' <br>
> print(n.zfill(3))

004

2. pad zeros to number
> n = 4 <br>
> print(f'{n:03}') # Preferred method, python >= 3.6

004

In [ ]:
def generateUrl(id):
  baseUrl = "https://api-customer.thaichana.com/shop/0001/S"
  maxSidLen = 10

  ##---manually pad zeros---
  #idLen = len(str(id))
  #numZero = maxSidLen - idLen
  #sId = '0' * numZero + str(id)

  if type(id) == str:
      sId = id.zfill(maxSidLen)
  elif type(id) == int:
      sId = str(id).zfill(maxSidLen)
  #print(sId)

  url = baseUrl + sId
  return url

In [ ]:
# test our function by giving integer as an argument
generateUrl(111)

'https://api-customer.thaichana.com/shop/0001/S0000000111'

In [ ]:
# test our function by giving string as an argument
generateUrl('3')

'https://api-customer.thaichana.com/shop/0001/S0000000003'

### Retrive info of multiple shops, using our two custom function

In [ ]:
# url = "https://api-customer.thaichana.com/shop/0001/S0000010001"
shops = []
for i in range(1,2001):
  url = generateUrl(i)
  print(url)
  data = getData(url)
  shops.append(data)
  #input()
shops

https://api-customer.thaichana.com/shop/0001/S0000000001


NameError: ignored

In [ ]:
### Let's print details of all shops that we have just gather
df = json_normalize(shops)
df

""


In [ ]:
df.isna()

""


In [ ]:
df.isna().sum()

Series([], dtype: float64)

In [ ]:
# Drop records with missing shopID Define in which columns to look for missing values.
df.dropna(subset=['shopId'], inplace=True)
df

KeyError: ignored

In [ ]:
df.isna().sum()

In [ ]:
df[df['shopName'].isna()]

KeyError: ignored

## delete rows from a pandas DataFrame based on a conditional expression

df = df.drop(some labels)

df = df.drop(df[<some boolean condition>].index)

Example

To remove all rows where column 'score' is < 50:

df = df.drop(df[df.score < 50].index)

In place version (as pointed out in comments)

df.drop(df[df.score < 50].index, inplace=True)

### Let's drop rows of shopId, whose info is missing.

In [ ]:
df.drop(df[df['shopName'].isna()].index, inplace=True)
df

In [ ]:
df.isna().sum()

In [ ]:
df[df['shopName'].isna()]

In [ ]:
df[df['businessType']=='สถาบันการเงิน']

In [ ]:
df[(df['businessType']=='ธนาคาร') & (df['shopName'].str[0:4]!='บมจ.')]

In [ ]:
df[(df['businessType']=='ธนาคาร') & (df['shopName'].str.find('กรุงไทย')==-1)]

In [ ]:
df[(df['businessType']=='ซุปเปอร์มาร์เก็ต') & (df['shopName'].str.find('เซ็นทรัล')!=-1)]

### Example 5 - Request from Pantip.com
#### try to get api from website that sen requests to get content for client side rendering.

ex: https://pantip.com/forum/supachalasai

In [ ]:
url = 'https://pantip.com/api/forum-service/forum/room_topic?room=supachalasai&limit=50&next_id=41598960'
header_dict = {'ptauthorize': 'Basic dGVzdGVyOnRlc3Rlcg=='} # modify here
response = requests.get(url, headers = header_dict)
data = response.json()
data

{'success': True,
 'data': [{'topic_id': 41598923,
   'title': 'ทำไงดีคะเมื่อรู้สึกท้อ',
   'topic_type': 3,
   'created_time': '2022-08-23T12:08:57Z',
   'views_count': 180,
   'comments_count': 2,
   'votes_count': 0,
   'author': {'id': 6653436,
    'name': 'สมาชิกหมายเลข 6653436',
    'avatar': {'original': '',
     'large': 'https://ptcdn.info/images/avatar_member_default.png',
     'medium': 'https://ptcdn.info/images/avatar_member_default.png',
     'small': 'https://ptcdn.info/images/avatar_member_default.png'},
    'slug': '/profile/6653436'},
   'tags': [{'name': 'กีฬา', 'slug': 'กีฬา'}]},
  {'topic_id': 41598897,
   'title': 'เฮนโด้มีส่วนแจกประตูลูกที่ 2 ให้แมนยู เหมือนตอนที่เจอร์ราดลื่นแล้วแจกประตูให้เชลซีหรือไม่ครับ',
   'topic_type': 3,
   'created_time': '2022-08-23T11:55:27Z',
   'views_count': 768,
   'comments_count': 4,
   'votes_count': 0,
   'author': {'id': 6426948,
    'name': 'สมาชิกหมายเลข 6426948',
    'avatar': {'large': 'https://ptcdn.info/images/avatar_memb


---


# Data Collection from Database (Ex. PostgreSQL)

**SQLAlchemy** is a convenient Python package to natively interact with databases. It can retrieve data and put into the Dataframe of Pandas directly. SQLAlchemy also suppots ORM, which boosts to development speed.
psycopg2 is a database driver to connect PostgreSQL.

In [ ]:
import pandas as pd
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
psycopg2.__version__

'2.9.3 (dt dec pq3 ext lo64)'

In [ ]:
sqlalchemy.__version__

'1.4.40'

In [ ]:
# Configuration of all required params to connect PostgreSQL DB
POSTGRES_HOST='34.87.67.47'
POSTGRES_DATABASE='ds4biz'
POSTGRES_USER='ds4biz_student'
POSTGRES_PASSWORD='lab-icbiz535'

In [ ]:
# Test Connection
def connect_db():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(
            host=POSTGRES_HOST,
            database=POSTGRES_DATABASE,
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD)

        # create a cursor
        cur = conn.cursor()

	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

def get_db_data(sql):

    conn = 'postgresql+psycopg2://' + POSTGRES_USER + ':' + POSTGRES_PASSWORD + '@' + POSTGRES_HOST + '/' + POSTGRES_DATABASE

    """ query data from the vendors table """
    dbConnection = None
    df = None
    try:
        # Create an engine instance
        alchemyEngine = create_engine(conn, pool_recycle=3600);

        # Connect to PostgreSQL server
        dbConnection = alchemyEngine.connect();

        # Read data from PostgreSQL database table and load into a DataFrame instance
        df = pd.read_sql(sql, dbConnection);

        pd.set_option('display.expand_frame_repr', False);

        # Close the database connection
        dbConnection.close();
    except (Exception) as error:
        print(error)
    finally:
        if dbConnection is not None:
            dbConnection.close()

    return df;

In [ ]:
# Test connection and its configuration
connect_db()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 13.3 (Debian 13.3-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)
Database connection closed.


## Entity Relationship Diagram for DVD Rental

<figure>
<center>
<img src="https://www.it.kmitl.ac.th/~teerapong/resources/ds4biz/week3/images/dvd-rental-ERD.png" width="700"/>
<figcaption><em>Search Page</em>
</figcaption></center>
</figure>

There are 15 tables in the DVD Rental database:

1.   actor – stores actors data including first name and last name.
2.   film – stores film data such as title, release year, length, rating, etc.
3.   film_actor – stores the relationships between films and actors.
4.   category – stores film’s categories data.
5.   film_category- stores the relationships between films and categories.
6.   store – contains the store data including manager staff and address.
7.   inventory – stores inventory data.
8.   rental – stores rental data.
9.   payment – stores customer’s payments.
10.   staff – stores staff data.
11.   customer – stores customer data.
12.   address – stores address data for staff and customers
13.   city – stores city names.
14.   country – stores country names.
15.   language - stores language names of audio in film.

In [ ]:
# Query customer's rental about how many time each converted customer rents movie DVDs from the store

sql = "SELECT c.customer_id, first_name, last_name, email, count(*) AS rental_time \
	      FROM public.rental AS r \
	      LEFT JOIN public.customer AS c \
	        ON r.customer_id = c.customer_id \
 	      GROUP BY c.customer_id \
        ORDER BY c.customer_id"

df = get_db_data(sql)
df

,customer_id,first_name,last_name,email,rental_time
0,1,Mary,Smith,mary.smith@sakilacustomer.org,32
1,2,Patricia,Johnson,patricia.johnson@sakilacustomer.org,27
2,3,Linda,Williams,linda.williams@sakilacustomer.org,26
3,4,Barbara,Jones,barbara.jones@sakilacustomer.org,22
4,5,Elizabeth,Brown,elizabeth.brown@sakilacustomer.org,38
...,...,...,...,...,...
594,595,Terrence,Gunderson,terrence.gunderson@sakilacustomer.org,30
595,596,Enrique,Forsythe,enrique.forsythe@sakilacustomer.org,28
596,597,Freddie,Duggan,freddie.duggan@sakilacustomer.org,25
597,598,Wade,Delvalle,wade.delvalle@sakilacustomer.org,22


In [ ]:
# Query customer's rental about how many time each converted customer rents movie DVDs from the store

sql = "SELECT first_name, last_name \
				FROM public.customer \
				WHERE first_name LIKE ''%u%''"

df1 = get_db_data(sql)
df1

'dict' object does not support indexing


In [ ]:
df[df['first_name']=='Austin']

,customer_id,first_name,last_name,email,rental_time
598,599,Austin,Cintron,austin.cintron@sakilacustomer.org,19


In [ ]:
df[(df['rental_time']>20) & (df['rental_time']<=25)]

,customer_id,first_name,last_name,email,rental_time
3,4,Barbara,Jones,barbara.jones@sakilacustomer.org,22
7,8,Susan,Wilson,susan.wilson@sakilacustomer.org,24
8,9,Margaret,Moore,margaret.moore@sakilacustomer.org,23
9,10,Dorothy,Taylor,dorothy.taylor@sakilacustomer.org,25
10,11,Lisa,Anderson,lisa.anderson@sakilacustomer.org,24
...,...,...,...,...,...
582,583,Marshall,Thorn,marshall.thorn@sakilacustomer.org,23
584,585,Perry,Swafford,perry.swafford@sakilacustomer.org,24
589,590,Seth,Hannon,seth.hannon@sakilacustomer.org,25
596,597,Freddie,Duggan,freddie.duggan@sakilacustomer.org,25


In [ ]:
# customer's rental in detail for each transaction and detail about movie

sql = "SELECT c.customer_id, first_name, last_name, email, f.title, rental_rate, rating, rental_date \
	      FROM public.rental AS r \
	      LEFT JOIN public.customer AS c \
		      ON r.customer_id = c.customer_id \
	      LEFT JOIN public.inventory as inv \
		      ON r.inventory_id = inv.inventory_id \
	      LEFT JOIN public.film as f \
		      ON inv.film_id = f.film_id \
        ORDER BY c.customer_id"

df = get_db_data(sql)
df

,customer_id,first_name,last_name,email,title,rental_rate,rating,rental_date
0,1,Mary,Smith,mary.smith@sakilacustomer.org,Adaptation Holes,2.99,NC-17,2005-08-01 08:51:04
1,1,Mary,Smith,mary.smith@sakilacustomer.org,Patient Sister,0.99,NC-17,2005-08-19 09:55:16
2,1,Mary,Smith,mary.smith@sakilacustomer.org,Bikini Borrowers,4.99,NC-17,2005-08-22 20:03:46
3,1,Mary,Smith,mary.smith@sakilacustomer.org,Attacks Hate,4.99,PG-13,2005-06-18 08:41:48
4,1,Mary,Smith,mary.smith@sakilacustomer.org,Musketeers Wait,4.99,PG,2005-06-15 00:54:12
...,...,...,...,...,...,...,...,...
16039,599,Austin,Cintron,austin.cintron@sakilacustomer.org,Blues Instinct,2.99,G,2005-08-23 11:25:00
16040,599,Austin,Cintron,austin.cintron@sakilacustomer.org,Family Sweet,0.99,R,2005-06-18 06:29:53
16041,599,Austin,Cintron,austin.cintron@sakilacustomer.org,Others Soup,2.99,PG,2005-07-12 21:23:59
16042,599,Austin,Cintron,austin.cintron@sakilacustomer.org,Zorro Ark,4.99,NC-17,2005-08-21 17:43:42
